In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import requests
import io 
#from transformers import pipeline # Might crash kernal needs lots of disk space and RAM

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.max_colwidth', None)


In [31]:
# Base URL for NYC GOV Jobs JSON data API
base_url = "https://data.cityofnewyork.us/resource/kpav-sd4t.json"

limit = 1000 
offset = 0 
all_data = []


while True:
   
    params = {
        "$limit": limit,
        "$offset": offset
    }
    
    response = requests.get(base_url, params=params)
    
    # If the request was successful, process the data
    if response.status_code == 200:
       
        chunk_data = response.json()
        
        if not chunk_data:
            break
        
        # Append the current chunk to the list
        all_data.extend(chunk_data)
        
        # Update the offset for the next request
        offset += limit
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        break

# Convert the list of dictionaries to a DataFrame
full_df = pd.DataFrame(all_data)

# Drop rows where 'Job ID' is NaN
full_df = full_df.dropna(subset=['job_id'])

# Show the first few rows of the cleaned data
print(full_df.head())


   job_id                          agency posting_type number_of_positions  \
0  695839  HOUSING PRESERVATION & DVLPMNT     Internal                   1   
1  590513  DEPT OF ENVIRONMENT PROTECTION     External                   1   
2  645999     HRA/DEPT OF SOCIAL SERVICES     External                   1   
3  585475   DEPT OF DESIGN & CONSTRUCTION     Internal                   1   
4  546011  DEPT OF ENVIRONMENT PROTECTION     External                   1   

                                                                           business_title  \
0  Family Self Sufficiency Case Manager for the Division of Program Policy and Innovation   
1                                                                        Project Engineer   
2                                                                         UNIT SUPERVISOR   
3                                                                  Junior Project Manager   
4                                                      Director of

In [32]:
full_df.columns

Index(['job_id', 'agency', 'posting_type', 'number_of_positions',
       'business_title', 'civil_service_title', 'title_classification',
       'title_code_no', 'level', 'job_category',
       'full_time_part_time_indicator', 'career_level', 'salary_range_from',
       'salary_range_to', 'salary_frequency', 'work_location',
       'division_work_unit', 'job_description', 'minimum_qual_requirements',
       'residency_requirement', 'posting_date', 'post_until',
       'posting_updated', 'process_date', 'preferred_skills',
       'additional_information', 'to_apply', 'work_location_1', 'hours_shift'],
      dtype='object')

In [33]:
# Display the first few rows of the DataFrame
#print(.head())

In [34]:
# only external postings
df = full_df
df = df[df['posting_type'] == 'External']

# Filter by desired poistions
titles_to_match = ['Data Scientist', 'Software Engineer', 'Data Analyst', 'Software Developer', 'Data', 'Software']
df = df[df['business_title'].str.contains('|'.join(titles_to_match), case=False, na=False)]

#df.head()


In [35]:
df.sort_values(by = 'posting_date', ascending=False, inplace=True)

df['posting_date'] = pd.to_datetime(df['posting_date'])
df['last_updated'] = pd.to_datetime(df['posting_updated'], errors='coerce')

# Get the date 4 months ago from today
four_months_ago = datetime.now() - timedelta(days=4*30)

# Filter the rows
df = df[~((df['posting_date'] <= four_months_ago) & df['last_updated'].isna())]


In [36]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
df['Job Description Summarized'] = summarizer(df['Job Description'], max_length=130, min_length=30)

#df['Job Description Summarized'].head(5)

NameError: name 'pipeline' is not defined

In [ ]:
cols = ['Job ID', '# Of Positions', 'Business Title', 'Career Level', 'Division/Work Unit', 
                  'Job Description Summarized', 'Minimum Qual Requirements', 'Preferred Skills', 
                  'Additional Information', 'To Apply', 'Posting Date', 'Post Until', 'Posting Updated'
]

cols_more_info = ['job_id', 'number_of_positions', 'business_title',  'job_category',
'full_time_part_time_indicator', 'career_level', 'salary_range_from',
'salary_range_to', 'division_work_unit', 'job_description', 
'minimum_qual_requirements', 'preferred_skills', 'additional_information', 'to_apply',
'posting_date', 'post_until', 'posting_updated'
]

# Display the first 50 rows for the specified columns
df[cols_more_info].head(50)


job_id number_of_positions  \
2946  700009                   1   
4680  699757                   1   
5006  698427                   1   
3573  698236                   1   
235   698615                   1   
4670  698153                   1   
4949  698455                   1   
5745  698705                   1   
595   698722                   1   
5602  697924                   1   
3992  697960                   1   
1698  697955                   1   
761   697870                   1   
3320  638244                   1   
3773  697331                   1   
3195  697329                   1   
2428  694819                   1   
1541  696351                   1   
5470  695990                   1   
2214  694728                   1   
3981  693146                   2   
4167  642518                   1   
3643  637280                   1   
789   689520                   1   
4081  691407                   1   
2331  691616                   1   
702   689682                   1   
193   689356                   1   
201   687799                   1   
170   685108                   1   
99    682179                   1   
2680  682438                   1   
4594  683058                   1   
5615  683454                   1   
4699  683587                   1   
4615  683484                   1   
2333  682556                   1   
1468  682607                   1   
5060  681921                   2   
3266  625852                   1   
5911  680965                   1   
2291  678919                   1   
4128  679854                   1   
122   678926                   2   
3930  649450                   1   
4378  657137                   1   
1204  647931                   1   
4529  647477                   1   
601   647654                   1   
1958  646522                   1   

                                                                                                                                                       business_title  \
2946                                                                           Director, Data Governance Strategy and Standards, Bureau of Data Technology & Strategy   
4680                                                                                                                                        Analyst  Data and Systems   
5006                                                                                       Contract Manager, Center for Population Health Data Science Administration   
3573                                                                                             Senior Research Data Scientist, Bureau of Data Technology & Strategy   
235                                                                                                                                                      Data Analyst   
4670  Senior Evaluator & Quality Analyst, Office for Data Aggregation, Translation, Analytics (ODATA), Community Awareness, Action, Response, and Engagement (BCAARE)   
4949                                                                                                                                          Data Analyst Specialist   
5745                                                                                                                    Data Analyst, Bureau of Epidemiology Services   
595                                                                                                    Data Analyst, Bureau of Environmental Sciences and Engineering   
5602                                                                                                                                       Data Analytics Grad Intern   
3992                                                                                                                                       Data Analytics Grad Intern   
1698                                                                                                                                       Da